<a href="https://colab.research.google.com/github/forhigh/deeplearning_undergrad/blob/main/%5Bcomplete%5Dlab_07_6_FieldCompaction_ValidationDataset_EarlyStopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_07_6 Deep Neural Nets
Field Compaction of Open-Graded Aggregates Validation Dataset
Early Stopping

환경 준비

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import pandas as pd
import numpy as np

데이터 준비, 정규화, Training/Testing 데이터셋 구분

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Data_set = np.loadtxt("/content/gdrive/My Drive/Colab Notebooks/FieldCompaction1.csv", delimiter=',', dtype=np.float32)
X = Data_set[:, 0:2]
Y = Data_set[:, 2:5]

# Normalize input dataset X
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

# Split dataset into training and testing datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


딥러닝 모델 구조 설정

In [3]:
model = Sequential()
model.add(Dense(30, input_dim=2, activation='relu'))
model.add(Dense(3))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                90        
                                                                 
 dense_1 (Dense)             (None, 3)                 93        
                                                                 
Total params: 183
Trainable params: 183
Non-trainable params: 0
_________________________________________________________________


모델 컴파일, 자동 중단 설정, 최적화 모델 저장 설정, 실행

In [4]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20) # 검증셋의 오차가 20회 이상 낮아지지 않을 경우 학습을 종료

# 최적화 모델이 저장될 폴더와 파일 이름 설정
modelpath = "/content/gdrive/My Drive/Colab Notebooks/Checkpoint/FieldCompaction-bestmodel.hdf5"

# 최적화 모델 저장 설정
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

# 모델 실행
history = model.fit(X_train, Y_train, epochs=5000, validation_split=0.25, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/5000
2/2 [==============================] - 1s 294ms/step - loss: 5441.7007 - mean_squared_error: 5441.7007 - val_loss: 4737.5586 - val_mean_squared_error: 4737.5586
Epoch 2/5000
2/2 [==============================] - 0s 53ms/step - loss: 5439.3101 - mean_squared_error: 5439.3101 - val_loss: 4735.3638 - val_mean_squared_error: 4735.3638
Epoch 3/5000
2/2 [==============================] - 0s 47ms/step - loss: 5436.9341 - mean_squared_error: 5436.9341 - val_loss: 4733.1865 - val_mean_squared_error: 4733.1865
Epoch 4/5000
2/2 [==============================] - 0s 45ms/step - loss: 5434.5605 - mean_squared_error: 5434.5605 - val_loss: 4731.0205 - val_mean_squared_error: 4731.0205
Epoch 5/5000
2/2 [==============================] - 0s 55ms/step - loss: 5432.1831 - mean_squared_error: 5432.1831 - val_loss: 4728.8623 - val_mean_squared_error: 4728.8623
Epoch 6/5000
2/2 [==============================] - 0s 46ms/step - loss: 5429.8696 - mean_squared_error: 5429.8696 - val_loss: 4726.71

테스트 결과 출력

In [5]:
score=model.evaluate(X_test, Y_test)
print('Test Loss:', score)

1/1 [==============================] - 0s 18ms/step - loss: 186.4052 - mean_squared_error: 186.4052
Test Loss: [186.40518188476562, 186.40518188476562]


history에 저장된 학습 결과 확인


In [6]:
hist_df=pd.DataFrame(history.history)
hist_df

,loss,mean_squared_error,val_loss,val_mean_squared_error
0,5441.700684,5441.700684,4737.558594,4737.558594
1,5439.310059,5439.310059,4735.363770,4735.363770
2,5436.934082,5436.934082,4733.186523,4733.186523
3,5434.560547,5434.560547,4731.020508,4731.020508
4,5432.183105,5432.183105,4728.862305,4728.862305
...,...,...,...,...
1123,228.771591,228.771591,180.673569,180.673569
1124,228.455490,228.455490,180.647079,180.647079
1125,228.088669,228.088669,180.622574,180.622574
1126,227.756546,227.756546,180.594009,180.594009
